In [1]:
import pandas as pd
import numpy as np
from plotly.graph_objs import *
from pyecharts import options as opts
from pyecharts.charts import Kline, Line

In [2]:
df = pd.read_csv("BTC_daily.csv")               # 可改为ETH_daily.csv
df["Close_1_shift"] = df["Close"].shift(1) 
K = 2                                           # K倍标准差
N = 20                                          # N日移动平均线

In [3]:
df["sma"] = np.round(df["Close"].rolling(N).mean(),4)
df["stdev"] = np.round(df["Close"].rolling(N).std(),4) 
df["upband"]  = np.round(df["sma"] + K*df["stdev"],4)
df["downband"] = np.round(df["sma"] - K*df["stdev"],4)

In [4]:
money = [1000.0 for i in range(20)]  # initial money value in USDT
crypto_value = [0 for i in range(20)]
crypto = 0 
fees = 0.001           # fees as 0.1%
positions = []  
success_history = []  
failure_history = []  

#布林通道顺势策略

for i in range(21, df.shape[0]):
    if crypto == 0 and df["Close"].iloc[i-1] > df["upband"].iloc[i-1] and i != df.shape[0] - 1:     # 1.突破上轨线，建仓

        price = (df["Close"].iloc[i] + df["Open"].iloc[i]) / 2.0
        crypto = np.round(money[-1]*(1.0 - fees)/ price, 4)
        crypto_value += [crypto*df["Close"].iloc[i]]
        money += [0]
        positions += [{ "date": df.Date.iloc[i], "price": price}]
        
    elif crypto == 0:                                                                    # 2.空仓不动
        money += [money[-1]]
        crypto_value += [0]
            
    elif crypto > 0 and (df["Close"].iloc[i-1] < df["downband"].iloc[i-1] or i == df.shape[0] - 1 ):   # 3.突破下轨线，清仓       
        
        price = (df["Close"].iloc[i] + df["Open"].iloc[i]) / 2.0
        money += [crypto * price *(1-fees)]
        crypto = 0
        crypto_value +=[0]
        
        if positions[-1]["price"] < price:
            success_history += [
                {
                    "date": [positions[-1]["date"], df.Date.iloc[i]],
                    "price": [positions[-1]["price"], price],
                }
            ]
        else:
            failure_history += [
                {
                    "date": [positions[-1]["date"], df.Date.iloc[i]],
                    "price": [positions[-1]["price"], price],
                }
            ]
        positions = []

    elif crypto > 0:                                                             # 4.持仓不动
        money += [0]
        crypto_value += [crypto * df["Close"].iloc[i]]

rtn1 = [np.round((crypto_value[i]+money[i])/money[0],4) for i in range(df.shape[0]-1)]


In [5]:
money = [1000.0 for i in range(20)]  # initial money value in USDT
crypto_value = [0 for i in range(20)]
crypto = 0 
fees = 0.001           # fees as 0.1%
positions = []  
success_history = []  
failure_history = []  

#布林通道逆势策略

for i in range(21, df.shape[0]):
    if crypto == 0 and df["Close"].iloc[i-1] < df["downband"].iloc[i-1] and i != df.shape[0] - 1:     # 1.突破下轨线，建仓

        price = (df["Close"].iloc[i] + df["Open"].iloc[i]) / 2.0
        crypto = np.round(money[-1]*(1.0 - fees)/ price, 4)
        crypto_value += [crypto*df["Close"].iloc[i]]
        money += [0]
        positions += [{ "date": df.Date.iloc[i], "price": price}]
        
    elif crypto == 0:                                                                    # 2.空仓不动
        money += [money[-1]]
        crypto_value += [0]
            
    elif crypto > 0 and (df["Close"].iloc[i-1] > df["upband"].iloc[i-1] or i == df.shape[0] - 1 ):   # 3.突破上轨线，清仓       
        
        price = (df["Close"].iloc[i] + df["Open"].iloc[i]) / 2.0
        money += [crypto * price *(1-fees)]
        crypto = 0
        crypto_value +=[0]
        
        if positions[-1]["price"] < price:
            success_history += [
                {
                    "date": [positions[-1]["date"], df.Date.iloc[i]],
                    "price": [positions[-1]["price"], price],
                }
            ]
        else:
            failure_history += [
                {
                    "date": [positions[-1]["date"], df.Date.iloc[i]],
                    "price": [positions[-1]["price"], price],
                }
            ]
        positions = []

    elif crypto > 0:                                                             # 4.持仓不动
        money += [0]
        crypto_value += [crypto * df["Close"].iloc[i]]


In [6]:
attr=[str(t) for t in df.Date]
rtn2 = [np.round((crypto_value[i]+money[i])/money[0],4) for i in range(df.shape[0]-1)]
ben = [np.round(j/df.Close.iloc[0],4) for j in df.Close]

line2 = (
    Line()
    .add_xaxis(attr)
    .add_yaxis("rtn_boll_1",rtn1 , is_smooth = True,)
    .add_yaxis("rtn_boll_2",rtn2 , is_smooth = True,)
    .add_yaxis("ben",ben , is_smooth = True,)
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(is_scale=True),
        yaxis_opts=opts.AxisOpts(
            is_scale=True,
            splitarea_opts=opts.SplitAreaOpts(
                is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
            ),
        ),
        datazoom_opts=[opts.DataZoomOpts(pos_bottom="-2%")],
        title_opts=opts.TitleOpts(title="boll_compare"),
    ) 
)
line2.render("boll_compare.html")

'F:\\jupyter\\boll_compare.html'